In [ ]:
# imports
import os
import time
import pandas as pd
from skimage.measure import label,regionprops
from skimage.morphology import binary_opening, erosion, dilation, ball
from skimage.segmentation import watershed
from scipy import ndimage
from sklearn.mixture import BayesianGaussianMixture
from skimage.morphology import opening, closing, ball
from skimage.filters import gaussian
from skimage.transform import rescale
from scipy.stats import entropy
import numpy as np
import nrrd
from skimage.segmentation import morphological_geodesic_active_contour as gac
from skimage.segmentation import inverse_gaussian_gradient as igg
from scipy.ndimage.morphology import binary_fill_holes
from scipy.stats import entropy
# from scripts.ImageSliceViewer3D import ImageSliceViewer3D as isv 

<div class="alert alert-block alert-info">
<b>Note:</b> 
    Functionally, the pipeline below uses the same methods for segmentation as those found in the public dataset segmentation notebook. The key difference is in the "semi" of the semi-automated pipeline. Here, we leverage user-defined bounding boxes (with ~3mm expansion) as a starting point for the pipeline. The method that accomplishes this is in the next cell. 

</div>

In [ ]:
''' 
SUPPLEMENTARY FUNCTION : createSubVolumesClinical
This function isolates the sub-volume of the image that we are interested in. 
This way, we can perform operations only on the pixels containing lesion and surrounding pixels (executes faster).
    
    INPUT:
            image         - the original CT volume;
            image_dict    - dictionary with CT metadata;
            mask          - the original mask with radiologist-defined ROIs;
            lprop         - regionprops object for the user-defined start point.
            
    OUTPUT:
            image_subV    - image sub-volume, which contains lesion plus surrounding voxels;
            mask_subV     - ground truth labels, which occupies same space as both image and cylinder sub-volumes.
            
'''

def createSubVolumesClinical(image,mask,lprop):
    
    # get the bounding box for the user-defined mask in image coordinates
    coords = lprop.bbox 
    row = 3  # take an extra 3 voxels on either side of bounding box for row and col
    slc = 1  # take an extra voxel on either side of bounding box for axial slices
    
    img = image.copy()
    msk = mask.copy()
    
    # crop the image and mask to the expanded bounding box
    image_subV = img[coords[0]-row:coords[3]+row,coords[1]-row:coords[4]+row,coords[2]-slc:coords[-1]+slc]
    mask_subV = msk[coords[0]-row:coords[3]+row,coords[1]-row:coords[4]+row,coords[2]-slc:coords[-1]+slc]
    
    return image_subV,mask_subV

In [ ]:
''' 
SUPPLEMENTARY FUNCTION : determineThreshold
This function determines a threshold separating candidate lesion voxels from surrounding lung paremchyma. 
It leverages the BayesianGaussianMixture functionality from sklearn. 
Since the pre-defined sub-volume contains both lesion and lung parenchyma pixels, density distribution 
of the pixels could be modeled by two Gaussian distributions: P(x|lesion) and P(x|parenchyma), 
where x was the pixel density. The mean values and variations of the two Gaussian distributions were 
then estimated by the expectation-maximization method.
    
    INPUT:
            image_subV - the CT sub-volume;
            mask_subV  - mask occupying the same space as the image_subV.
            
    OUTPUT:
            threshold     - floating point threshold (anything above this threshold is a candidate lesion voxel)
                     
'''

def determineThreshold(image_subV,mask_subV):
    
    # isolate the intensities within the mask
    intensities = image_subV[mask_subV]

    # apply Gaussian Mixture model to obtain biologically-grounded threshold
    classif = BayesianGaussianMixture(n_components=2)
    classif.fit(intensities.reshape(-1,1))

    return np.mean(classif.means_)

In [ ]:
''' 
SUPPLEMENTARY FUNCTION : resampleVolumes
This function resamples the sub-volume and corresponding mask to isotropic voxel size (1mm, 1mm, 1mm). 
    
    INPUT:
            image     - the CT sub-volume;
            mask      - the mask occupying the same space as the image_subV;
            meta      - dictionary with original image and/or mask metadata (they are the same so either dictionary works).
            
    OUTPUT:
            img_RS    - the resampled CT sub-volume;
            msk_RS    - the resampled mask occupying the same space as the resampled CT sub-volume.
                     
'''

def resampleVolumes(image,mask,meta):

    dims = [meta['space directions'][0][0],meta['space directions'][0][0],meta['space directions'][-1][-1]]

    scaleFac = []
            
    for d in dims:
        if d < 1:
            scaleFac.append(1/float(d))
        else:
            scaleFac.append(float(d))
 
    # note the logical expression for the mask b/c the output from rescale is not binary
    msk_RS = rescale(mask, scale = (scaleFac[0],scaleFac[1],scaleFac[2]), preserve_range = True, anti_aliasing=None) > 0.5
    img_RS = rescale(image, scale = (scaleFac[0],scaleFac[1],scaleFac[2]), preserve_range = True, anti_aliasing=True)
            
    return img_RS,msk_RS

In [ ]:
''' 
SUPPLEMENTARY FUNCTION : calcStats
This function isolates sub-regions of the segmented lesion (core, periphery, etc.) and calculates statistics of interest. 

    INPUT:
            image     - the resampled CT sub-volume;
            mask      - the resampled mask occupying the same space as the image_subV;
            r         - optional argument (specified to 2mm, which is appropriate for the resampled sub-volume).
            
    OUTPUT:
            stats     - statistics of interest for the sub-regions of interest (mean intensity and entropy).
                     
'''

def prepArray(arr):
    
    # get the range of arr
    r = np.ptp(arr)
    hist, bin_edges = np.histogram(arr,bins=10,density=True)
    
    return hist


def calcStats(image,mask,r=2):
    
    # put the masks into a list
    all_masks = []
    
    # stats 
    stats = []
    
    # (0) lesion
    all_masks.append(mask)
    
    # (1) lesion core
    all_masks.append(erosion(mask,ball(radius = r)))
    
    # (2) lesion interior rim
    all_masks.append(np.logical_and(mask,~erosion(mask,ball(radius = r))))
    
    # (3) lesion exterior rim
    all_masks.append(np.logical_and(~mask,dilation(mask,ball(radius = r))))
    
    # iterate through the list of masks to calculate the stats
    # CODE HERE
    for i in range(len(all_masks)):
        
        if ~all_masks[i].any():
            stats.append(np.nan)
            stats.append(np.nan)
        else:
            stats.append(np.mean(image[all_masks[i]]))
            stats.append(entropy(prepArray(image[all_masks[i]])))
            
    if np.nanmean(stats[0:-1:2]) > 0:
  
        for i in range(len(stats)):
            if i%2 == 1:
                continue
            else:
                stats[i] = stats[i] - 1024
        
        
    return stats

In [ ]:
# lists of image/mask files
img_path = '/Users/EL-CAPITAN-2016/OneDrive - UHN/SARC/timeseries/semi-automated/Images'
msk_path = '/Users/EL-CAPITAN-2016/OneDrive - UHN/SARC/timeseries/semi-automated/Masks'

all_images = sorted([os.path.join(img_path,f) for f in os.listdir(img_path) if 'nrrd' in f])
all_masks = sorted([os.path.join(msk_path,f) for f in os.listdir(msk_path) if 'nrrd' in f])

<div class="alert alert-block alert-info">
<b>NOTE:</b>
    
The `all_images` and `all_masks` variables contain pointers to individual files. The files in these folders are not separated by time point. Take the first four pointers in the `all_images` variable as an example:
<br>   
`./Images/112001_BL_image.nrrd`,<br>
`./Images/112001_C2_image.nrrd`,<br>
`./Images/112002_BL_image.nrrd`,<br>
`./Images/112002_C2_image.nrrd`,...<br>
<br>   
The first six digits represent a unique subject identifier, and the two letters that follow encode the timepoint, where **BL** is the baseline or first measurement and **C2** is the measurement after 2 cycles of chemotherapy or second measurement.
</div>

In [ ]:
# first pass of segmentation pipeline
def segPipeline(imgList,mskList):
    
    # import the trial arm key -- the arm that received the hypoxia drug is encoded as 1
    key = pd.read_csv('./trial_arm_key.csv')
    stats_columns = ['lesion mu','lesion S','core mu','core S','interior mu','interior S','exterior mu','exterior S']
    
    # initialize lists for segmentation accuracy
    usubjid = []
    timepoint = []
    trial_arm = []
    lesion_index = []
    lesion_volume = []
    stats = []
    error_log = []
    algo_skip = []
    
    for i in range(len(imgList)):
        
        # read image and corresponding mask
        img_V,img_d = nrrd.read(imgList[i])
        msk_V,msk_d = nrrd.read(mskList[i])
        
        # calculate image and mask voxel volumes
        patient_id,time_id,fileID = os.path.basename(imgList[i]).split('_')
        msk_voxel_volume = msk_d['space directions'][0][0]**2 * msk_d['space directions'][-1][-1] # in mm^3
        img_voxel_volume = img_d['space directions'][0][0]**2 * img_d['space directions'][-1][-1] # in mm^3
        
        # if the image and mask do not have the same voxel volume, do not process
        if abs(msk_voxel_volume - img_voxel_volume) > 0.0001:
            print('ERROR, {}: image and mask must have same voxel volumes'.format(imgList[i]))
            error_log.append(imgList[i])
            continue
            
        # label the mask into connected regions
        lesion_labels,num_lesions = label(msk_V,return_num=True)
        lesion_props = regionprops(lesion_labels)

        # track lesion number
        lesion_count = 0
        
        # for every lesion in the mask
        for obj in lesion_props:
            
            usubjid.append(patient_id)
            timepoint.append(time_id)
            trial_arm.append(key.loc[key['USUBJID'] == int(patient_id),'ARM'].item())
            lesion_index.append(lesion_count)
            lesion_count += 1
            
            # create the sub-volume of interest within the CT volume
            img_subV,msk_subV = createSubVolumesClinical(img_V,msk_V,obj)
            msk_subV = msk_subV > 0
            
            # calculate threshold using EM
            threshold = determineThreshold(img_subV,msk_subV)
            
            # label the cylinder sub-volume into connected regions
            msk_subV_labels = label(msk_subV)
            msk_subV_props = regionprops(msk_subV_labels)
            
            # get the dimensions of the user-defined mask
            bbox = msk_subV_props[0].bbox  
            bbox_dims = [bbox[3]-bbox[0],bbox[4]-bbox[1],bbox[-1]-bbox[2]]
            R = max(bbox_dims) / 2 

            # coordinates for the user-defined lesion centroid
            msk_j,msk_i,msk_k = [round(i) for i in msk_subV_props[0].centroid]

            # apply the threshold
            binary_img = np.logical_and(np.logical_and(img_subV > threshold,img_subV > -850),img_subV < 200)
            binary_img[~msk_subV] = False
            
            # construct a marker image for the watershed
            marker_image = np.zeros((img_subV.shape[0],img_subV.shape[1],img_subV.shape[2]),dtype=np.uint8)
            marker_image[~binary_img] = 2         # background/lung parenchyma voxels
            marker_image[msk_j,msk_i,msk_k] = 1   # user-defined lesion centroid

            # denoise image sub-volume
            denoised = gaussian(img_subV,multichannel=False)

            # compute the morphological gradient
            max_image = dilation(denoised,ball(2))
            min_image = erosion(denoised,ball(2))
            gradient_image = max_image - min_image
            gradient_image[~msk_subV] = np.min(gradient_image)

            # create distance matrix for the 3D bowl function
            row,col,slc = np.meshgrid(range(gradient_image.shape[1]),range(gradient_image.shape[0]),range(gradient_image.shape[2]))
            dist_matrix = np.sqrt((round(msk_j) - row)**2 + (round(msk_i) - col)**2 + ((round(msk_k) - slc) * msk_d['space directions'][-1][-1])**2)
            dist_matrix[dist_matrix>=R] = R
            dist_matrix = dist_matrix / R
            
            # modify the gradient image for watershed
            mod_gradient = gradient_image * dist_matrix
            
            # perform watershed segmentation
            water_initial = ~watershed(mod_gradient, marker_image,connectivity=2)
            
            # label the watershed mask into connected regions
            water_labels = label(water_initial,background = np.min(water_initial))
            water_props = regionprops(water_labels)
            
            # find the largest region
            water_areas = [water_props[i].area for i in range(len(water_props))]
            ind = np.where(water_areas == np.max(water_areas))[0][0]

            # create the mask
            water_mask = water_labels == water_props[ind].label
            
            # omitted
            # FINALLY this initial segmentation is fed into the active contours function for further refinement
#             refined_mask = gac(igg(img_subV), iterations = 3, init_level_set=water_mask, threshold = 0.5)
#             refined_mask = gac(igg(img_subV), iterations = 15, init_level_set=binary_img, threshold = 0.5,balloon = -2)
            # fill any holes in the final mask result (unlikely, but you never know)
#             refined_mask = binary_fill_holes(refined_mask)

            v = water_mask.sum() * msk_voxel_volume
    
            # if the volume is less than 25% of what the user defined, the algorithm clearly failed :(
            if v < (msk_subV.sum() * msk_voxel_volume) / 4:
                
                # resample the volume to apply the morphological operations
                img_RS,msk_RS = resampleVolumes(img_subV,msk_subV,img_d)
                # take the core of the user-defined volume as the segmentation when all else fails
                user_mask = erosion(msk_RS,ball(radius=2))
                
                v = user_mask.sum()
                lesion_volume.append(v)
                
                # calculate the average intensity and entropy for all regions
                stats.append(calcStats(img_RS,user_mask))
                # make note of the image where the segmentation didn't work for future debugging
                algo_skip.append(imgList[i])
                
            else:
                lesion_volume.append(v)
                # resample the volume to apply the morphological operations
                img_RS,msk_RS = resampleVolumes(img_subV,water_mask,img_d)
                # calculate the average intensity and entropy for all regions
                stats.append(calcStats(img_RS,msk_RS))

            output_dict = { 'USUBJID' : usubjid,         # patient identifier (6-digit number)
                            'TIMEPT'  : timepoint,       # timepoint identifier (2-letter code -- BL baseline, C2 cycle 2)
                            'ARM'     : trial_arm,       # trial arm (0 control group, 1 experimental group)
                            'LSNIND'  : lesion_index,    # lesion index (for patients with more than 1 lesion)
                            'LSNVOL'  : lesion_volume}   # lesion volume (in mm^3)
    
        # progress report for the end-user  :)
        print('Processed image {}/{}: {}% complete.'.format(i+1,len(imgList),float(i+1)/len(imgList)*100))
        # concatenate the volume and stats information
        frames = [pd.DataFrame(output_dict),pd.DataFrame(stats,columns=stats_columns)] 
    
    return pd.concat(frames,axis=1),error_log,algo_skip
        

# run the pipeline
start_time = time.time()
df,error_log,algo_skip = segPipeline(all_images,all_masks)
print('Total processing time: {:.2f} minutes.'.format((time.time()-start_time)/60))